In [3]:
import pandas as pd
from sas7bdat import SAS7BDAT
import numpy as np
from datetime import date
import datetime as DT
from datetime import datetime
import io

## Create one dataframe with all information

In [4]:
df_tumour = pd.read_csv("C:\Data\crc_tumor.csv")
df_tumour["tum_incidentie_dtm"] = pd.to_datetime(df_tumour['tum_incidentie_dtm'])

df_patient = pd.read_csv("C:\Data\crc_patient.csv")
df_patient["pat_geboorte_dtm"] = pd.to_datetime(df_patient['pat_geboorte_dtm'])
df_patient["pat_overlijden_dtm"] = pd.to_datetime(df_patient['pat_overlijden_dtm'])
df_patient["pat_laatste_contact_dtm"] = pd.to_datetime(df_patient['pat_laatste_contact_dtm'])

df_cp = pd.read_csv("C:\Data\crc_ziektebeeld.csv")

df_treatment = pd.read_csv("C:\Data\crc_behandeling.csv")
df_treatment["gbs_eind_dtm"] = pd.to_datetime(df_treatment["gbs_eind_dtm"])
df_treatment["gbs_begin_dtm"] = pd.to_datetime(df_treatment["gbs_begin_dtm"])

In [5]:
# From treatment to patient (patient (rn)-> ziektebeeld (zid)-> tumor (eid)-> behandeling)
df_tum_tr = pd.merge(df_tumour, df_treatment, on=["eid","rn","zid"])
df_tum_tr_cp = pd.merge(df_tum_tr, df_cp, on=["rn","zid"])
df_all = pd.merge(df_tum_tr_cp, df_patient, on=["rn"])

cols_to_order = ["rn","zid","eid", "gid"]
new_columns = cols_to_order + (df_all.columns.drop(cols_to_order).tolist())
df = df_all[new_columns]

## Only keep colon carcinoma and one record per patient (and only if it is first episode)

In [6]:
# Select all records with cancer colon carcinoma (tumourtype 205310) and keep one record per rn/patient (first episode and then first incidence date)
df['tum_tumorsoort_id'] = df['tum_tumorsoort_id'].apply(np.int64)
df_cc = df[df['tum_tumorsoort_id'] == 205310]
df_cc = df_cc[df_cc['eerste_episode'] == 1] # 1 means yes first episode and 0 means no not first episode (because of years of survival)
# Keeping only one record per patient
df_cc = df_cc.sort_values('tum_incidentie_dtm')
df_cc = df_cc.groupby('rn').first().reset_index()

### Create dataset with 17 features

In [7]:
df_cc1 = df_cc[['rn', 'pat_geboorte_dtm', 'pat_geslacht_code', 'tum_incidentie_dtm', 'tum_topo_sublokalisatie_code', 'tum_morfologie_code', 
                'tum_differentiatiegraad_code', 'vitstat', 'ct', 'cn', 'cm', 'pt', 'pn', 'pm', 'stadium', 'tum_lymfklieren_onderzocht_atl',
              'tum_lymfklieren_positief_atl', 'pat_overlijden_dtm']]

### Create dataset with 9 features

In [8]:
df_cc2 = df_cc[['rn', 'pat_geboorte_dtm', 'pat_geslacht_code', 'tum_incidentie_dtm', 'tum_topo_sublokalisatie_code', 'tum_differentiatiegraad_code',
                'vitstat', 'stadium', 'tum_lymfklieren_positief_atl', 'pat_overlijden_dtm']]

## Save both subsets as csv file

In [9]:
df_cc1.to_csv('C:\Users\bde2002.53381\Desktop\master-thesis\experiments\data\input\df1_subset.csv', index=False)
df_cc2.to_csv('C:\Users\bde2002.53381\Desktop\master-thesis\experiments\data\input\df2_subset.csv', index=False)